# MS 141 Lab 2. Errors in Numerical Calculations
### Read: Chapter 4 Newman. Optional but recommended: Chapters 2 and 3 of Landau's book.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Set common figure parameters
newparams = {'figure.figsize': (10, 5), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

# Types of Error

Whether you are careful or not, errors and uncertainties are part of computation.<br>
There are several types or errors in computations, and each requires different strategies to be identified and addressed.

## Manual errors
The include bugs in the code or its subroutines or libraries (occasionally, the compiler or hardware may be responsible) and can be difficult to identify.<br> 
Always be suspicious of your code and treat results as incorrect unless proven otherwise. If possible, validate your code on a case with known analytic solution.<br>

Move from simple examples to more complex cases. If possible, ask other team members to read your code.<br> 
The *production run* leading to publication-quality results is always the last step of the method development and testing process.<br> 
It should always be generated with a code that has been benchmarked and validated.<br> 

Community efforts lead to better validated code that in-house developed codes used only in one project. For this reason, using existing and previously validated code (at least as a starting point for developing new tools) may be desirable in many cases. <br><br>

## Approximation (truncation) errors 

Imprecision due to mathematical approximations introduced to treat a problem numerically.<br> 
Typical examples include approximating an integral with a summation, a derivative with a finite difference, or an infinite series with a finite sum.<br>

For example, a function can be expanded in Taylor series as:<br>

$$ f(x+h) = f(x) + f'(x)\,h + f''(x)\,h^2/2 + \ldots$$

If we truncate the expansion at the term of order $h^{\,n}$, the error will be of order $h^{\,n+1}$, which may or may not be acceptable depending on the value of $h$ and the problem at hand. This type of problem comes up when approximating derivatives and integrals, as we'll see later in this course.

## Round-off errors 

Errors arising due to the limited precision with which computers represent and store **real and complex numbers**. They can add up during calculations.<br>

### Number representation
**Integers.** A computer represents numbers in memory using binary digits. Each number corresponds to a string of 0 and 1. If we represent integers with 16 bits, <br> we can only represent $2^{16} = 65536$ integers. If we want both positive and negative integers, they will range from $-32768$ to $+32767$.<br> 
This range is unacceptably small for scientific computing, so we typically represent integers using 32 bits (type `int` in python) with approximate range $\pm\, 2 \cdot 10^9$ or 64 bits (type `long`, a long integer in python), with approximate range $\pm\, 9 \cdot 10^{18}$.<br>

In principle, if one attempts to add 1 to the maximum 64 bit integer (9223372036854775807), the result should lead to *overflow*. Depending on the system architecture, the number would either not change, or the system would restart counting from the smallest integer, or something else.<br> 
In C++ and Fortran, integer overflow can occur and it can be a problem that should be kept in mind.<br> 
In Python 3, integers are implemented as 64 bit "long" integers, but they are made of arbitrarily large size (within reason) to avoid overflow.

In [ ]:
# Let's test integer overflow in Python
import sys

i = sys.maxsize
print(i)
# will print 9223372036854775807

print(i == i + 1)
# False: no overflow

i += 1
print(i)
# will not overflow, and print 9223372036854775808

**Real numbers (and complex numbers).** Real numbers can take any value, including irrational ones. It would take an arbitrarily large amount of memory<br> to represent most rational and all irrational numbers with arbitrary precision. Another challenge in scientific computing is that we want to represent both<br> *very small* and *very large* numbers, ideally with the same precision. A trade-off between range and precision is needed.<br>
Note that complex numbers are pairs or reals, so the same considerations made here for reals apply also to complex numbers.<br>

The solution is to represent real numbers as **floating point numbers (floats)**, storing separately the sign, decimal digits, and exponent of a real number.<br> 
A real number is written as 

$$ x_{\,\rm float} = (-1)^{sign} \times 1.\,f \times 2^{\,\rm exponent} $$

where $f$ is the decimal part (the so-called fractional mantissa).<br> 
The mantissa sets the numerical (machine) precision, and thus the *round-off error*, while the exponent sets the largest and smallest numbers one can represent.

The IEEE sets a standard for how to represent real numbers using 32 bits (single-precision floats) or 64 bits (double precision floats).<br> 
In our experience, **scientific computing always requires using double-precision floats**. Python uses double-precision floats by default.<br>

In [double-precision floats](https://en.wikipedia.org/wiki/Double-precision_floating-point_format), the 64 bits are used as follows: 1 bit for the sign, 52 bits for the mantissa, and 11 bits for the exponent.<br>
As a result, real numbers are written as:

$$ (-1)^{\text{sign}}(1.b_{51}b_{50}...b_{0})_{2}\times 2^{e-1023} $$
or equivalently
$$(-1)^{\text{sign}}\left(1+\sum _{i=1}^{52}b_{52-i}2^{-i}\right)\times 2^{e-1023}.$$

Let us parse this information. The exponent $e$ is stored using 11 bits, so it takes values between $0$ and $2^{11} = 2048$. We want to represent both small and large numbers, so we choose to represent exponents ranging between +1024 and -1023 (by biasing the exponent). The largest real one can represent with double floats is of order $2^{1023} \approx 10^{308}$, and the smallest is of order $10^{-324}$. In python 3, *floats will overflow and underflow*, which is something to keep in mind.<br>

The mantissa sets the machine precision. For double floats, the smallest decimal number one can write is one part in $2^{52}$, that is, $2^{-52} \approx 4.5 \times 10^{-16}$.<br>
We say that double floats have *16 significant digits*. In practice, this means that an error should be expected in the $15^{th}$ digit in most cases.<br> Note also that errors combine and can add up during calculations. The result of a computation can be trusted only to at most 15$-$16 digits.<br> 
It does not make sense to look for higher accuracy or, say, provide as input to a computer program a number with more than 16 digits.<br> 
We say that $\varepsilon_m \approx 10^{-16}$ is the *machine precision.*

Let us see these concepts in action with a few examples.<br><br>

In [ ]:
# check float overflow
f = sys.float_info.max
print(f)
# 1.7976931348623157e+308

if (f == f + 1):
    print ('\nOverflow!\n')
# True

f += 1
print(f)
# 1.7976931348623157e+308

In [ ]:
# check float underflow
f = sys.float_info.min
print(f)
# prints 2.2250738585072014e-308

# however, I can still use the mantissa to write ~10^-16
# this leads to a lower limit of about 5*10^-324 for floats

print (f* 1.e-15)  # no underflow
print (f* 1.2e-16) # almost underflow
print (f* 1.e-16)  # underflow!

In [ ]:
# Determine your machine precision (within a factor of two)
# We can define it as the largest number which, added to the number stored as 1, still gives 1 as a result.

N = 53 # try 53
eps = 1.0 # machine precision

for i in range(N):
    eps = eps/2
    one_plus_eps = 1.0 + eps
    print ('eps = ', eps)
    print ('one + eps = ' , one_plus_eps, '\n')

### Round-off errors in numerical calculations

There are several implications of finite machine precision and the associated round-off errors in practical numerical calculations.<br> 
An excellent discussion on this topic is in Chapter 3 of Landau's book.<br> 

**Round-off error accumulation.** 
In a typical calculation, round-off errors will *add up randomly*. In this scenario, if the precision of a number is $\varepsilon$, after $N$ operations one should expect a lower precision of $\sqrt{N}\, \varepsilon$. 
This result implies that an algorithm with a large number of steps may lead to large round-off errors.<br>
In some cases, such as recursive algorithms, the error may not cancel randomly; it may add up as $N \varepsilon$ (or worse) after $N$ steps, dominating the numerical result.

**Addition and subtraction.** 
Another implication is that one should try to add numbers starting from the smallest to the largest to retain as many significant digits as possible (100.x has only 13 significant decimal digits, while 1.x has 15). 
Another well-known issue is the *subtractive cancellation error*. One should never subtract two large and nearly equal numbers as the result would have fewer significant digits than the original numbers. 

**Truncation vs. round-off error.** Truncation and round-off error have opposite trends. Truncation errors can be decreased by adding more terms or using more computational steps, while round-off error increases with increasing number of steps. Therefore, an ideal algorithm is one that can obtain an accurate answer with a relatively small number of steps to avoid accumulating round-off error.<br> 
Note that for error analysis, the most important quantity is the **relative error $\frac{\Delta a}{ a}$** rather than the absolute error $\Delta a$.<br>

**Convergence.** Converging a numerical result may require adding many terms or carrying out many steps. Typically, a result is converged if it will not change appreciably when adding a few more terms or carrying out a few more steps. One should be careful about not adding too many terms as the error will grow again due to round-off error accumulation. 

### Example 1: Addition in floating-point arithmetic
The order in which numbers are added in the summation $\sum_{n=1}^N \frac{1}{n}$ is important when using floating point arithmetic.<br> 
In general, the downward sum, in which numbers are added from the smallest to the largest, leads to more accurate results.

In [ ]:
# Example 1: Order of number addition in the summation sum_{n=1}^N (1/n)
# Page 57 in Landau's book
import numpy as np

N = np.array([10, 100, 500, int(1e4), int(1e5), int(1e6), int(5e6), int(1e7)])
length = len(N)
r = np.zeros(len(N))

# These two summations should in principle give the same result:
# upward sum
for i in range(len(N)):
    s_up = 0.
    s_down = 0.

    for n in range(1,N[i]+1):
        # debug: print (n)
        s_up += 1/n

    # downward sum (will be larger)
    # why? See the answer by Godric Seer at 
    # https://scicomp.stackexchange.com/questions/10869
    for m in range(N[i],0,-1):
        #debug: print (m)
        s_down += 1/m
    
    print ('Added ', N[i], ' terms')
    print ('up:   ', s_up)
    print ('down: ', s_down, '\n')

    r [i] = (s_up - s_down) #/(abs(s_up) + abs(s_down))

print (r)

In [ ]:
# plot the results

plt.plot(N,-r,'--b', marker='o',markersize=10,markerfacecolor='blue')
plt.xscale('log')
plt.yscale('log')

plt.xlim(10,50000000)
plt.xlabel('N')
plt.ylabel('sum_down - sum_up')

plt.show()

### Example 2: Subtractive cancellation error
Let us consider a variable $x$ represented as $x_c$ inside the computer. The relative error for the variable $x$ is 
$ \varepsilon_x = \frac{x_c - x}{x} $,<br> and thus the numerical representation can be written as $x_c = x(1+\varepsilon_x)$. We expect $\varepsilon_x$ to be close in value to the machine precision.<br>

Consider subtracting two numbers $x$ and $y$, with result $s = x - y$. How is the relative error in $s$ related to the errors in $x$ and $y$?<br>
Inside the computer, $s_c = x_c - y_c \approx x(1+\varepsilon_x) - y(1+\varepsilon_y)$, and thus the relative error in the subtraction is:<br>

$$ \frac{s_c - s}{s} = \frac{x}{s}\varepsilon_x - \frac{y}{s}\varepsilon_y.$$

There is no assurance that the two terms in the righthand side will cancel.<br> 
When we subtract two nearly equal large numbers $x \approx y$, we get that $s$ is small compared to $x$ and $y$, which leads to a large relative error because we are subtracting off the most significant parts of both numbers and leaving the most error-prone least significant digits. Numerically:

$$ \frac{s_c - s}{s} \approx \frac{x}{s} (\varepsilon_x - \varepsilon_y) \approx \frac{x}{s} \max(|\varepsilon_x|, |\varepsilon_y|).$$

Even if the relative error in $x$ and $y$ cancel somewhat, the result is multiplied by the large number $\frac{x}{s}$, leading to loss of significant digits. Because we cannot assume any sign for the errors, we assume the worst case scenario (the "max" in the equation above). 

An example is the summation of alternating series in which the sign of consecutive terms alternates from positive to negative. Two such cases are 

$$\sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \ldots =\, 
\sum_{n=1}^\infty \frac{(-1)^{n-1} x^{2n-1} }{(2n-1)!}$$ 

and 

$$ e^{\,-x} = 1 - x + \frac{x^2}{2!} - \frac{x^3}{3!} + \ldots =\, \sum_{n=0}^\infty \frac{(-1)^n\, x^n}{n!} $$

If these series are computed as finite summations (up to $N$ terms), when $x$ is large the first few terms give large subtractive cancellation errors,<br> and the accuracy of the summation is compromised. 
Let us explore $e^{\,-x}$ numerically.

In [ ]:
import numpy as np

# alternating series up to N terms
def altsum(N):
    my_sum = 0.0
    for n in range(N):
        my_sum += (-1)**n * x**n / float(np.math.factorial(n))
    return my_sum

# pick a "large" value, x = 10 
# the first few terms give large cancellation errors
x = 10.0 
exact = np.exp(-x)

N = 35 # sum N terms
approx = altsum(N)
print (exact - approx)

# as you'll see, there is a terrible loss of accuracy
# we need N=100 (and thus N>>x) to obtain a reasonable result

In [ ]:
# compute e^-x as 1/e^x 
def psum(N): #positive sum
    my_sum = 0.0
    for n in range(N):
        my_sum += x**n / float(np.math.factorial(n)) #e^x
    return 1/my_sum # 1/e^x

N = 35 
approx = psum(N)
print (exact - approx)

# the problem goes away
# N=35 already gives close to machine accuracy

## Truncation vs. round-off errors

Truncation and round-off errors typically exhibit competing trends. Truncation error requires more terms to be eliminated, while round-off error increases as more terms are combined together.<br> 

In a well-designed algorithm, convergence is relatively rapid, and the truncation error dies off as a power-law of the number of steps or terms computed: $\varepsilon_t \,\approx\, \alpha \,/\, N^\beta$, with $\alpha$ a constant and $\beta$ a positive integer. On the other hand, the round-off error increases with $N$, typically as the square root: $\varepsilon_r \,\approx\, \sqrt{N}\,\varepsilon_m$,
where $\varepsilon_m$ is the machine error.

Assuming $\alpha = 1$ and $\beta = 2$ for the sake of the argument, the total error is (note: Landau's book, page 65, has errors which have been fixed here):

$$ \varepsilon_{\rm tot} \,=\, \frac{1}{N^2} + N^{1/2}\,\varepsilon_m $$ 

The error is minimal when $d\varepsilon_{\rm tot}\,/\,dN = 0$, with solution $N^{5/2} = 4 \,/\, \varepsilon_m$. For double-precision floats, taking $\varepsilon_m = 10^{-15}$, we get an optimal number of steps or terms $N = 1741101$, for which $\varepsilon_{\rm tot} \approx 1.65 \times 10^{-12}$. In this case, most of the error is round-off rather than truncation error.

For a faster truncation error decay, $\varepsilon_t = 2/N^4$, one would find an optimal value $N = 3989$ with a total error $\varepsilon_{\rm tot} \approx 6.7 \times 10^{-14}$. This is a better algorithm, which requires much fewer steps with less build-up of round-off error. The result is more accurate by a factor of $\sim25$ for a small fraction of the steps.<br>
This is the type of algorithm one should strive for.

### Example 3. Optimal increment in the forward derivative

Let us investigate truncation vs. round-off error in the computation of the derivative of a function at a point.<br>
We approximate the derivative at a point $x_0$ as

$$ f'(x) \,\approx\, \frac{f(x+h) - f(x)}{h}$$

for various values of the increment $h$. This so-called forward derivative is not a very good approximation as the truncation error is of order $h^2$.

In [ ]:
import numpy as np

# f(x) = x^2 + 1
def f(x):
    return x**2 + 1

# f'(1) = 2
x0 = 1.0
der_exact = 2*x0  

# compute f'(x0) as ( f(x0+h) - f(x0) ) / h
# for various values of the increment h
h = np.array([0.1,0.001,1.e-5,1.e-7, 1.e-9,1.e-11, 1.e-13, 1.e-15,1.e-17])
der = (f(x0+h) - f(x0))/h

# compute the relative error
rel = abs(der - der_exact)/der_exact
print (rel)

In [ ]:
# plot the error

plt.plot(h,rel,'--b', marker='o',markersize=10,markerfacecolor='blue')
plt.yscale('log')
plt.xscale('log')

plt.xlabel('Increment h')
plt.ylabel('Relative error')

plt.show()

We find that the error is minimal for a range of values of $h$ between $10^{-11}$ to $10^{-7}$. If we choose an increment value that is too small and too close to the machine precision, the error starts increasing as round-off error dominates. If we choose an increment that is too large, the truncation error of order $h^2$ dominates. 